<a href="https://colab.research.google.com/github/JuanDiaz77/Proyecto-colab/blob/main/S4S4_3_Desarrollo_y_despliegue_de_un_modelo_de_IA_aplicado_a_un_caso_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %% [markdown]
# Proyecto: Clasificador de Sentimientos (NLP) — Notebook Colab
#
# **Resumen / Objetivo**
# Construir un proyecto completo de IA en dominio de Procesamiento de Lenguaje Natural (NLP):
# - Selección del caso de uso: clasificación de sentimientos en reseñas (IMDB).
# - Preparación de datos, diseño y entrenamiento de modelo (DistilBERT), evaluación y despliegue básico con Gradio.
#

# %% [markdown]
# 1) Justificación de la elección de caso de uso
# - Dominio: NLP (procesamiento de texto). Relevancia práctica: análisis de sentimiento es útil para monitorizar opiniones de clientes, moderación de contenido y priorización.
# - Dataset público: IMDB (reseñas de películas, binario: positivo/negativo). Fácil de replicar y adecuado para demostrar todo el flujo de ML/AI.

# 2) Instalación de librerías (Colab)
# %%

# Ejecutar esta celda al inicio para instalar dependencias necesarias.

# 2. Instalación de librerías
!pip install -q transformers datasets gradio scikit-learn matplotlib

# %% [markdown]
# 3) Importaciones

# %%
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          Trainer, TrainingArguments)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import seaborn as sns

# %% [markdown]
# 4) Configuración reproducible

# %%
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# %% [markdown]
# 5) Carga del dataset (Hugging Face `datasets`)

# %%
print("Cargando dataset IMDB desde Hugging Face...")
dataset = load_dataset("imdb")

# Mostrar un ejemplo
print(dataset)
print('\nEjemplo de la entrada de entrenamiento:')
print(dataset['train'][0])

# %% [markdown]
# 6) Exploración rápida

# %%
# Conteo de clases
train_df = pd.DataFrame(dataset['train'])
print(train_df['label'].value_counts())

# Mostrar longitudes de texto resumen
train_df['length'] = train_df['text'].str.len()
print(train_df['length'].describe())

# %% [markdown]
# 7) Tokenización / Preprocesamiento
# Usamos DistilBERT (suficientemente rápido para una demo) — tokenizer y modelo desde Hugging Face.

# %%
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenización en batch. Ajuste `max_length` según memoria (ej. 128 o 256).
MAX_LENGTH = 128

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=MAX_LENGTH)

# Aplicar tokenización (nota: esto puede tardar en CPU/Colab; si se quiere más rápido, seleccionar un subset).
print('Tokenizando (esto tarda unos minutos)...')

# Para demo rápida: usar subset pequeño de entrenamiento y evaluación
TRAIN_SUBSET = 2000  # ajustar según tiempo y GPU/TPU disponible
EVAL_SUBSET = 500

# tokenizar y seleccionar subset
small_train = dataset['train'].shuffle(seed=RANDOM_SEED).select(range(TRAIN_SUBSET))
small_test = dataset['test'].shuffle(seed=RANDOM_SEED).select(range(EVAL_SUBSET))

tokenized_train = small_train.map(tokenize_function, batched=True)
tokenized_test = small_test.map(tokenize_function, batched=True)

# Especificar columnas para Trainer
tokenized_train = tokenized_train.remove_columns([c for c in tokenized_train.column_names if c not in ['input_ids','attention_mask','label']])
tokenized_test = tokenized_test.remove_columns([c for c in tokenized_test.column_names if c not in ['input_ids','attention_mask','label']])

# Convertir a formato de datasets que Trainer acepta (ya lo es)

# %% [markdown]
# 8) Cargar el modelo preentrenado para clasificación

# %%
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# %% [markdown]
# 9) Métricas para evaluación

# %%
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

# %% [markdown]
# 10) Configuración y entrenamiento con Trainer

# %%
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Corrected argument name
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy='no'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entrenamiento (puede tardar más si no hay GPU)
print('Entrenando... (esto puede tardar)')
trainer.train()

# %% [markdown]
# 11) Evaluación final

# %%
eval_result = trainer.evaluate()
print('Resultados de evaluación:')
print(eval_result)

# Predicciones sobre el conjunto de test reducido
preds_output = trainer.predict(tokenized_test)
preds = np.argmax(preds_output.predictions, axis=1)
labels = preds_output.label_ids

# Matriz de confusión
cm = confusion_matrix(labels, preds)
print('Matriz de confusión:\n', cm)

# %% [markdown]
# 12) Visualización: matriz de confusión

# %%
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicho')
plt.ylabel('Verdadero')
plt.title('Matriz de confusión (test subset)')
plt.show()

# %% [markdown]
# 13) Análisis de errores (ejemplos)

# %%
wrong_idx = np.where(preds != labels)[0]
print(f'Total errores en subset: {len(wrong_idx)} — mostramos hasta 5 ejemplos')
for i in wrong_idx[:5]:
    print('\n--- Ejemplo ---')
    print('Label:', labels[i])
    print('Pred:', preds[i])
    print('Text snippet:', small_test[i]['text'][:400])

# %% [markdown]
# 14) Despliegue básico con Gradio
# - Creamos una función predictora ligera que usa el tokenizer y el modelo entrenado.
# - En Colab, `demo.launch(share=True)` genera una URL pública temporal.

# %%
import gradio as gr

def clasificar_sentimiento(texto: str):
    # Preprocesar y predecir
    inputs = tokenizer(texto, return_tensors='pt', truncation=True, padding=True, max_length=MAX_LENGTH)
    # Pasar a dispositivo (si GPU está disponible)
    model_inputs = {k: v for k, v in inputs.items()}
    outputs = model(**model_inputs)
    pred = int(outputs.logits.argmax(-1).item())
    label = 'Positiva' if pred == 1 else 'Negativa'
    return label

# Interfaz Gradio
iface = gr.Interface(fn=clasificar_sentimiento,
                     inputs=gr.Textbox(lines=4, placeholder='Escribe una reseña...'),
                     outputs='text',
                     title='Clasificador de Sentimientos (DistilBERT)',
                     description='Introduce una reseña y predice si es positiva o negativa.')

# Para lanzar en Colab: ejecutar iface.launch(share=True)
print('Para lanzar la interfaz Gradio en Colab: ejecutar: iface.launch(share=True)')

# %% [markdown]
# 15) Conclusiones técnicas y reflexivas (plantilla)
# - Resultados: incluir métricas impresas en la celda de evaluación.
# - Fortalezas: uso de modelo preentrenado, pipeline reproducible, despliegue rápido con Gradio.
# - Limitaciones: subset para entrenamiento (por tiempo/memoria), posible overfitting, necesidad de ajuste de hiperparámetros y mayor preprocesamiento (p. ej. limpieza de HTML, normalización).
# - Mejoras propuestas: entrenar con más datos, realizar búsqueda de hiperparámetros, usar técnicas de data augmentation para texto, probar modelos más grandes o fine-tuning con mayor budget, y desplegar en nube (Cloud Run / AWS / Hugging Face Spaces) para producción.

# %% [markdown]
# 16) Revisión final y entrega


# %% [markdown]
# FIN DEL NOTEBOOK


Cargando dataset IMDB desde Hugging Face...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: deve77opmint (deve77opmint-universidad-nacional-de-colombia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
